In [1]:
import torch
from models import FcEncoder, CnnEncoder, DensenetEncoder, Decoder
from utils import num_of_params

In [2]:
fc_encoder = FcEncoder()
cnn_encoder = CnnEncoder()
densenet_encoder = DensenetEncoder()
print(num_of_params(fc_encoder))
print(num_of_params(cnn_encoder))
print(num_of_params(densenet_encoder))


12845312
5519120
7216256


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class N(nn.Module):
    def __init__(self):
        super(N, self).__init__()
        self.net = nn.Linear(5, 3)
        self.net2 = nn.Linear(3, 5)

    def forward(self, x):
        return self.net2(F.relu(self.net(x)))

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available(): 
    device = torch.device("mps")
else:
    device = torch.device("cpu")

x = torch.Tensor([[0,1,0,1,0], [1,2,3,4,5], [9,8,7,6,5]])

encoder = N().to(device)
optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)
criterion = nn.MSELoss(reduction="sum")
def train(encoder, x):
    x = x.to(device)
    optimizer.zero_grad()
    code = encoder(x)
    loss = criterion(input=code, target=x) / len(x)

    loss.backward()               # Backward pass (compute parameter gradients)
    optimizer.step()              # Update weight parameter u

    return loss

In [12]:
for i in encoder.parameters():
    print(i)

Parameter containing:
tensor([[-0.0193,  0.0336,  0.2760,  0.1890,  0.2739],
        [ 0.3120, -0.0334, -0.0826,  0.3450, -0.3762],
        [-0.3210, -0.2503, -0.2984,  0.3961,  0.2285]], device='mps:0',
       requires_grad=True)
Parameter containing:
tensor([-0.3722, -0.4151, -0.3475], device='mps:0', requires_grad=True)
Parameter containing:
tensor([[ 0.3984,  0.5422,  0.3263],
        [-0.5589, -0.5321,  0.2227],
        [ 0.4160, -0.0032,  0.2223],
        [ 0.5436,  0.2981,  0.0540],
        [ 0.2001,  0.3407,  0.2692]], device='mps:0', requires_grad=True)
Parameter containing:
tensor([ 0.4633, -0.3991, -0.0479,  0.4381, -0.2393], device='mps:0',
       requires_grad=True)


In [13]:
train(encoder, x)
for i in encoder.parameters():
    print(i)

Parameter containing:
tensor([[-0.0183,  0.0346,  0.2770,  0.1900,  0.2749],
        [ 0.3110, -0.0344, -0.0836,  0.3440, -0.3772],
        [-0.3200, -0.2493, -0.2974,  0.3971,  0.2295]], device='mps:0',
       requires_grad=True)
Parameter containing:
tensor([-0.3712, -0.4161, -0.3465], device='mps:0', requires_grad=True)
Parameter containing:
tensor([[ 0.3994,  0.5432,  0.3253],
        [-0.5579, -0.5311,  0.2237],
        [ 0.4170, -0.0022,  0.2233],
        [ 0.5446,  0.2991,  0.0550],
        [ 0.2011,  0.3417,  0.2702]], device='mps:0', requires_grad=True)
Parameter containing:
tensor([ 0.4643, -0.3981, -0.0469,  0.4391, -0.2383], device='mps:0',
       requires_grad=True)


Images are well-labeled, exactly from 000001 to 202599. No gaps in between

In [13]:
import pandas as pd
labels = pd.read_csv("data/list_attr_celeba.csv", index_col="image_id")

In [15]:
labels.dtypes

5_o_Clock_Shadow       int64
Arched_Eyebrows        int64
Attractive             int64
Bags_Under_Eyes        int64
Bald                   int64
Bangs                  int64
Big_Lips               int64
Big_Nose               int64
Black_Hair             int64
Blond_Hair             int64
Blurry                 int64
Brown_Hair             int64
Bushy_Eyebrows         int64
Chubby                 int64
Double_Chin            int64
Eyeglasses             int64
Goatee                 int64
Gray_Hair              int64
Heavy_Makeup           int64
High_Cheekbones        int64
Male                   int64
Mouth_Slightly_Open    int64
Mustache               int64
Narrow_Eyes            int64
No_Beard               int64
Oval_Face              int64
Pale_Skin              int64
Pointy_Nose            int64
Receding_Hairline      int64
Rosy_Cheeks            int64
Sideburns              int64
Smiling                int64
Straight_Hair          int64
Wavy_Hair              int64
Wearing_Earrin

In [16]:
s = pd.Series([10, 20, 30], index=['a', 'b', 'c'])
s['a']

10

In [16]:
file = 'data/output/FC/0/features.pt'
tensor = torch.load(file)
tensor.shape

torch.Size([50000, 256])